Machine Learning

In [2]:
import requests
import pickle
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [3]:
#import pickle

#with open('match_df', 'rb') as f:
#    match_df = pickle.load(f)

In [4]:
match_df = pd.read_csv("match.csv", index_col=0)

In [5]:
matches = match_df.reset_index()
del matches['index']
matches.index = matches.index + 1


In [6]:
matches["verein"].value_counts()

Bayer Leverkusen       68
Bayern Munich          68
Koln                   68
Bochum                 68
Mainz 05               68
Union Berlin           68
Wolfsburg              68
Monchengladbach        68
Werder Bremen          68
Hoffenheim             68
Augsburg               68
Freiburg               68
Eintracht Frankfurt    68
Dortmund               68
RB Leipzig             68
Stuttgart              68
Heidenheim             34
Darmstadt 98           34
Schalke 04             34
Hertha BSC             34
Name: verein, dtype: int64

In [7]:
matches.dtypes

datum              object
uhrzeit            object
wett               object
runde              object
tag                object
spielort           object
ergebnis           object
tf                float64
tk                float64
gegner             object
xg                float64
xga               float64
besitz            float64
zuschauerzahl     float64
kapitän            object
aufstellung        object
schiedsrichter     object
spielbericht       object
hinweise          float64
sc                float64
sat               float64
entf.             float64
fs                float64
elf               float64
veelf             float64
saison              int64
verein             object
dtype: object

In [8]:
matches["datum"] = pd.to_datetime(matches["datum"],dayfirst=True)


In [9]:
matches["venue_code"] = matches["spielort"].astype("category").cat.codes

In [10]:
matches["opp_code"] = matches["gegner"].astype("category").cat.codes

In [11]:
#matches['datum'] = pd.to_datetime(matches['datum'])


In [12]:
matches2 = matches[(matches['datum'] > '2024-04-22') & (matches['datum'] < '2024-05-10')]

In [13]:
print(matches2)

         datum uhrzeit        wett          runde  tag  spielort ergebnis  tf  \
31  2024-04-27   18:30  Bundesliga  Spielwoche 31  Sa.      Heim      NaN NaN   
32  2024-05-05   17:30  Bundesliga  Spielwoche 32  So.  Auswärts      NaN NaN   
65  2024-04-27   15:30  Bundesliga  Spielwoche 31  Sa.      Heim      NaN NaN   
66  2024-05-04   15:30  Bundesliga  Spielwoche 32  Sa.  Auswärts      NaN NaN   
99  2024-04-27   18:30  Bundesliga  Spielwoche 31  Sa.  Auswärts      NaN NaN   
100 2024-05-04   15:30  Bundesliga  Spielwoche 32  Sa.      Heim      NaN NaN   
133 2024-04-27   15:30  Bundesliga  Spielwoche 31  Sa.      Heim      NaN NaN   
134 2024-05-03   20:30  Bundesliga  Spielwoche 32  Fr.  Auswärts      NaN NaN   
167 2024-04-27   15:30  Bundesliga  Spielwoche 31  Sa.  Auswärts      NaN NaN   
168 2024-05-04   15:30  Bundesliga  Spielwoche 32  Sa.      Heim      NaN NaN   
201 2024-04-27   15:30  Bundesliga  Spielwoche 31  Sa.  Auswärts      NaN NaN   
202 2024-05-05   17:30  Bund

In [14]:
matches = matches[~(matches['datum'] > '2024-04-22')]


In [15]:
print(matches.head(50))

        datum uhrzeit        wett          runde  tag  spielort ergebnis   tf  \
1  2023-08-19   15:30  Bundesliga   Spielwoche 1  Sa.      Heim        S  3.0   
2  2023-08-26   18:30  Bundesliga   Spielwoche 2  Sa.  Auswärts        S  3.0   
3  2023-09-02   15:30  Bundesliga   Spielwoche 3  Sa.      Heim        S  5.0   
4  2023-09-15   20:30  Bundesliga   Spielwoche 4  Fr.  Auswärts        U  2.0   
5  2023-09-24   15:30  Bundesliga   Spielwoche 5  So.      Heim        S  4.0   
6  2023-09-30   15:30  Bundesliga   Spielwoche 6  Sa.  Auswärts        S  3.0   
7  2023-10-08   15:30  Bundesliga   Spielwoche 7  So.      Heim        S  3.0   
8  2023-10-21   15:30  Bundesliga   Spielwoche 8  Sa.  Auswärts        S  2.0   
9  2023-10-29   17:30  Bundesliga   Spielwoche 9  So.      Heim        S  2.0   
10 2023-11-04   15:30  Bundesliga  Spielwoche 10  Sa.  Auswärts        S  3.0   
11 2023-11-12   15:30  Bundesliga  Spielwoche 11  So.      Heim        S  4.0   
12 2023-11-25   15:30  Bunde

In [16]:
matches["time"] = matches["uhrzeit"].str.replace(":.+", "", regex=True).astype("int")

In [17]:
matches["day_code"] = matches["datum"].dt.dayofweek

In [18]:
def transform(outcome):
    if outcome == "N":
        return 0
    elif outcome == "U":
        return 1
    elif outcome == "S":
        return 2

    

In [19]:
matches["target"] = matches["ergebnis"].apply(transform)

In [20]:
matches["target"] = matches["target"].astype(int)

Random Forest

In [21]:
rf = RandomForestClassifier

In [22]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [23]:
train = matches[matches["datum"]< "2024-01-01"]

In [24]:
test = matches[matches["datum"]> "2024-01-01"]

In [25]:
predictors = ["venue_code", "opp_code","time"]

In [26]:
rf.fit(train[predictors],train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [27]:
preds = rf.predict(test[predictors])

In [28]:
acc = accuracy_score(test["target"], preds)

In [29]:
acc

0.44881889763779526

In [30]:
combined = pd.DataFrame(dict(actual=test["target"], prediction = preds))

In [31]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1,2
actual,,,
0,56,5,30
1,30,11,31
2,31,13,47


In [32]:
precision_score(test["target"],preds,average='micro')

0.44881889763779526

Next Part

In [33]:
grouped_matches = matches.groupby("verein")

In [34]:
group = grouped_matches.get_group("Bayer Leverkusen")

In [35]:
group.head(5)

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,...,fs,elf,veelf,saison,verein,venue_code,opp_code,time,day_code,target
1,2023-08-19,15:30,Bundesliga,Spielwoche 1,Sa.,Heim,S,3.0,2.0,RB Leipzig,...,0.0,0.0,0.0,2024,Bayer Leverkusen,1,14,15,5,2
2,2023-08-26,18:30,Bundesliga,Spielwoche 2,Sa.,Auswärts,S,3.0,0.0,M'Gladbach,...,0.0,0.0,0.0,2024,Bayer Leverkusen,0,12,18,5,2
3,2023-09-02,15:30,Bundesliga,Spielwoche 3,Sa.,Heim,S,5.0,1.0,Darmstadt 98,...,1.0,0.0,0.0,2024,Bayer Leverkusen,1,3,15,5,2
4,2023-09-15,20:30,Bundesliga,Spielwoche 4,Fr.,Auswärts,U,2.0,2.0,Bayern München,...,1.0,1.0,1.0,2024,Bayer Leverkusen,0,1,20,4,1
5,2023-09-24,15:30,Bundesliga,Spielwoche 5,So.,Heim,S,4.0,1.0,Heidenheim,...,0.0,1.0,1.0,2024,Bayer Leverkusen,1,7,15,6,2


In [36]:
def rolling_averages(group,cols, new_cols):
    group = group.sort_values("datum")
    rolling_stats = group[cols].rolling(3, closed="left").mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [37]:
cols = ["tf", "tk", "sc", "sat", "entf.", "fs", "elf", "veelf"]
new_cols = [f"{c}_rolling" for c in cols]

In [38]:
new_cols

['tf_rolling',
 'tk_rolling',
 'sc_rolling',
 'sat_rolling',
 'entf._rolling',
 'fs_rolling',
 'elf_rolling',
 'veelf_rolling']

In [39]:
rolling_averages(group, cols, new_cols)

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,...,day_code,target,tf_rolling,tk_rolling,sc_rolling,sat_rolling,entf._rolling,fs_rolling,elf_rolling,veelf_rolling
786,2022-08-27,15:30,Bundesliga,Spielwoche 4,Sa.,Auswärts,S,3.0,0.0,Mainz 05,...,5,2,0.333333,2.000000,15.333333,5.333333,189.000000,1.333333,0.000000,0.000000
787,2022-09-03,15:30,Bundesliga,Spielwoche 5,Sa.,Heim,N,2.0,3.0,Freiburg,...,5,0,1.333333,1.666667,15.333333,6.000000,185.000000,1.666667,0.000000,0.000000
788,2022-09-10,15:30,Bundesliga,Spielwoche 6,Sa.,Auswärts,U,2.0,2.0,Hertha BSC,...,5,1,1.666667,2.000000,12.333333,4.666667,180.333333,1.333333,0.000000,0.000000
789,2022-09-17,15:30,Bundesliga,Spielwoche 7,Sa.,Heim,U,1.0,1.0,Werder Bremen,...,5,1,2.333333,1.666667,11.333333,5.333333,171.000000,0.666667,0.000000,0.000000
790,2022-09-30,20:30,Bundesliga,Spielwoche 8,Fr.,Auswärts,N,0.0,4.0,Bayern München,...,4,0,1.666667,2.000000,15.000000,6.000000,158.000000,0.333333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,2024-03-17,15:30,Bundesliga,Spielwoche 26,So.,Auswärts,S,3.0,2.0,Freiburg,...,6,2,2.000000,0.333333,17.666667,6.666667,187.333333,1.333333,0.000000,0.000000
27,2024-03-30,15:30,Bundesliga,Spielwoche 27,Sa.,Heim,S,2.0,1.0,Hoffenheim,...,5,2,2.333333,0.666667,18.333333,6.666667,176.666667,1.333333,0.000000,0.000000
28,2024-04-06,15:30,Bundesliga,Spielwoche 28,Sa.,Auswärts,S,1.0,0.0,Union Berlin,...,5,2,2.333333,1.000000,24.000000,8.666667,177.000000,1.000000,0.000000,0.000000
29,2024-04-14,17:30,Bundesliga,Spielwoche 29,So.,Heim,S,5.0,0.0,Werder Bremen,...,6,2,2.000000,1.000000,24.000000,8.333333,172.333333,1.000000,0.333333,0.333333


In [40]:
matches_rolling = matches.groupby("verein").apply(lambda x: rolling_averages(x,cols,new_cols))

In [41]:
matches_rolling = matches_rolling.droplevel("verein")

In [42]:
matches_rolling.index = range(matches_rolling.shape[0])

In [43]:
def make_predictions(data, predictors):
    train = data[data["datum"]< "2024-01-01"]
    test = data[data["datum"]> "2024-01-01"]
    rf.fit(train[predictors],train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted = preds), index = test.index)
    precision = precision_score(test["target"],preds,average='micro')
    return combined, precision

In [44]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [45]:
precision

0.4015748031496063

In [46]:
combined

,actual,predicted
47,0,2
48,2,0
49,0,2
50,1,0
51,1,2
...,...,...
1087,0,2
1088,2,0
1089,0,2
1090,0,0


In [47]:
combined = combined.merge(matches_rolling[["datum", "verein", "gegner", "ergebnis"]], left_index=True, right_index=True)

In [48]:
combined

,actual,predicted,datum,verein,gegner,ergebnis
47,0,2,2024-01-13,Augsburg,Leverkusen,N
48,2,0,2024-01-21,Augsburg,M'Gladbach,S
49,0,2,2024-01-27,Augsburg,Bayern München,N
50,1,0,2024-02-03,Augsburg,Bochum,U
51,1,2,2024-02-10,Augsburg,RB Leipzig,U
...,...,...,...,...,...,...
1087,0,2,2024-03-16,Wolfsburg,Augsburg,N
1088,2,0,2024-03-30,Wolfsburg,Werder Bremen,S
1089,0,2,2024-04-07,Wolfsburg,M'Gladbach,N
1090,0,0,2024-04-13,Wolfsburg,RB Leipzig,N


In [49]:
combined["verein"].unique()

array(['Augsburg', 'Bayer Leverkusen', 'Bayern Munich', 'Bochum',
       'Darmstadt 98', 'Dortmund', 'Eintracht Frankfurt', 'Freiburg',
       'Heidenheim', 'Hoffenheim', 'Koln', 'Mainz 05', 'Monchengladbach',
       'RB Leipzig', 'Stuttgart', 'Union Berlin', 'Werder Bremen',
       'Wolfsburg'], dtype=object)

In [50]:
combined["gegner"].unique()

array(['Leverkusen', "M'Gladbach", 'Bayern München', 'Bochum',
       'RB Leipzig', 'Mainz 05', 'Freiburg', 'Darmstadt 98', 'Heidenheim',
       'Wolfsburg', 'Köln', 'Hoffenheim', 'Union Berlin',
       'Eint Frankfurt', 'Augsburg', 'Werder Bremen', 'Dortmund',
       'Stuttgart'], dtype=object)

In [51]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Bayer Leverkusen":'Leverkusen',
    'Bayern Munich': 'Bayern München',
    'Eintracht Frankfurt': 'Eint Frankfurt',
    'Koln': 'Köln',
    'Monchengladbach': "M'Gladbach"
}
mapping = MissingDict(**map_values)


In [52]:
combined["new_team"] = combined["verein"].map(mapping)

In [53]:
merged = combined.merge(combined, left_on=["datum","new_team"],right_on=["datum","gegner"])

In [54]:
merged

,actual_x,predicted_x,datum,verein_x,gegner_x,ergebnis_x,new_team_x,actual_y,predicted_y,verein_y,gegner_y,ergebnis_y,new_team_y
0,0,2,2024-01-13,Augsburg,Leverkusen,N,Augsburg,2,0,Bayer Leverkusen,Augsburg,S,Leverkusen
1,2,0,2024-01-21,Augsburg,M'Gladbach,S,Augsburg,0,0,Monchengladbach,Augsburg,N,M'Gladbach
2,0,2,2024-01-27,Augsburg,Bayern München,N,Augsburg,2,0,Bayern Munich,Augsburg,S,Bayern München
3,1,0,2024-02-03,Augsburg,Bochum,U,Augsburg,1,0,Bochum,Augsburg,U,Bochum
4,1,2,2024-02-10,Augsburg,RB Leipzig,U,Augsburg,1,0,RB Leipzig,Augsburg,U,RB Leipzig
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,0,2,2024-03-16,Wolfsburg,Augsburg,N,Wolfsburg,2,2,Augsburg,Wolfsburg,S,Augsburg
250,2,0,2024-03-30,Wolfsburg,Werder Bremen,S,Wolfsburg,0,2,Werder Bremen,Wolfsburg,N,Werder Bremen
251,0,2,2024-04-07,Wolfsburg,M'Gladbach,N,Wolfsburg,2,0,Monchengladbach,Wolfsburg,S,M'Gladbach
252,0,0,2024-04-13,Wolfsburg,RB Leipzig,N,Wolfsburg,2,2,RB Leipzig,Wolfsburg,S,RB Leipzig


In [55]:
#merged[(merged["predicted_x"] == 2)& (merged["predicted_y"] == 0 )]["actual_x"].value_counts()

In [56]:
#28/49

In [57]:
merged.head(50)

,actual_x,predicted_x,datum,verein_x,gegner_x,ergebnis_x,new_team_x,actual_y,predicted_y,verein_y,gegner_y,ergebnis_y,new_team_y
0,0,2,2024-01-13,Augsburg,Leverkusen,N,Augsburg,2,0,Bayer Leverkusen,Augsburg,S,Leverkusen
1,2,0,2024-01-21,Augsburg,M'Gladbach,S,Augsburg,0,0,Monchengladbach,Augsburg,N,M'Gladbach
2,0,2,2024-01-27,Augsburg,Bayern München,N,Augsburg,2,0,Bayern Munich,Augsburg,S,Bayern München
3,1,0,2024-02-03,Augsburg,Bochum,U,Augsburg,1,0,Bochum,Augsburg,U,Bochum
4,1,2,2024-02-10,Augsburg,RB Leipzig,U,Augsburg,1,0,RB Leipzig,Augsburg,U,RB Leipzig
5,0,0,2024-02-17,Augsburg,Mainz 05,N,Augsburg,2,2,Mainz 05,Augsburg,S,Mainz 05
6,2,2,2024-02-25,Augsburg,Freiburg,S,Augsburg,0,0,Freiburg,Augsburg,N,Freiburg
7,2,0,2024-03-02,Augsburg,Darmstadt 98,S,Augsburg,0,2,Darmstadt 98,Augsburg,N,Darmstadt 98
8,2,2,2024-03-09,Augsburg,Heidenheim,S,Augsburg,0,0,Heidenheim,Augsburg,N,Heidenheim
9,2,2,2024-03-16,Augsburg,Wolfsburg,S,Augsburg,0,2,Wolfsburg,Augsburg,N,Wolfsburg


Insert new Data


In [58]:
matches_predict = matches2

In [59]:
matches_predict["time"] = matches_predict["uhrzeit"].str.replace(":.+", "", regex=True).astype("int")

In [60]:
matches_predict["day_code"] = matches["datum"].dt.dayofweek

In [61]:
matches_predict["target"] = matches["ergebnis"].apply(transform)

In [62]:
matches_predict["target"] = matches["target"].astype(int)

In [63]:
matches_predict.head()

,datum,uhrzeit,wett,runde,tag,spielort,ergebnis,tf,tk,gegner,...,fs,elf,veelf,saison,verein,venue_code,opp_code,time,day_code,target
31,2024-04-27,18:30,Bundesliga,Spielwoche 31,Sa.,Heim,NaN,NaN,NaN,Stuttgart,...,NaN,NaN,NaN,2024,Bayer Leverkusen,1,16,18,NaN,NaN
32,2024-05-05,17:30,Bundesliga,Spielwoche 32,So.,Auswärts,NaN,NaN,NaN,Eint Frankfurt,...,NaN,NaN,NaN,2024,Bayer Leverkusen,0,5,17,NaN,NaN
65,2024-04-27,15:30,Bundesliga,Spielwoche 31,Sa.,Heim,NaN,NaN,NaN,Eint Frankfurt,...,NaN,NaN,NaN,2024,Bayern Munich,1,5,15,NaN,NaN
66,2024-05-04,15:30,Bundesliga,Spielwoche 32,Sa.,Auswärts,NaN,NaN,NaN,Stuttgart,...,NaN,NaN,NaN,2024,Bayern Munich,0,16,15,NaN,NaN
99,2024-04-27,18:30,Bundesliga,Spielwoche 31,Sa.,Auswärts,NaN,NaN,NaN,Leverkusen,...,NaN,NaN,NaN,2024,Stuttgart,0,11,18,NaN,NaN


In [64]:
# Assuming 'rf' is your trained RandomForest model
if hasattr(rf, 'feature_names_in_'):
    print("Features used in training:")
    for feature in rf.feature_names_in_:
        print(feature)
else:
    print("The model does not have the 'feature_names_in_' attribute. Ensure it was trained with a pandas DataFrame.")

Features used in training:
venue_code
opp_code
time
tf_rolling
tk_rolling
sc_rolling
sat_rolling
entf._rolling
fs_rolling
elf_rolling
veelf_rolling


In [65]:
matches_predict2 = matches_predict.reindex(columns = matches_predict.columns.tolist() + ['tf_rolling',
 'tk_rolling',
 'sc_rolling',
 'sat_rolling',
 'entf._rolling',
 'fs_rolling',
 'elf_rolling',
 'veelf_rolling'])


In [66]:
columns_to_keep = ["venue_code", "opp_code","time", "tf_rolling","tk_rolling","sc_rolling","sat_rolling","entf._rolling","fs_rolling","elf_rolling","veelf_rolling"]


In [67]:
matches_predict4 = matches_predict2[columns_to_keep]

In [68]:
matches_predict4 = matches_predict4.fillna(0)

In [69]:
predictions = rf.predict(matches_predict4)

In [70]:
predictions

array([2, 1, 2, 0, 1, 2, 2, 1, 0, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2,
       2, 0, 1, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 1])

In [71]:
matches_predict2['predictions'] = predictions


In [72]:
df_reduziert = matches_predict2[["datum",	"uhrzeit", "gegner",  	"verein","predictions"]]


In [73]:
df_reduziert


,datum,uhrzeit,gegner,verein,predictions
31,2024-04-27,18:30,Stuttgart,Bayer Leverkusen,2
32,2024-05-05,17:30,Eint Frankfurt,Bayer Leverkusen,1
65,2024-04-27,15:30,Eint Frankfurt,Bayern Munich,2
66,2024-05-04,15:30,Stuttgart,Bayern Munich,0
99,2024-04-27,18:30,Leverkusen,Stuttgart,1
100,2024-05-04,15:30,Bayern München,Stuttgart,2
133,2024-04-27,15:30,Dortmund,RB Leipzig,2
134,2024-05-03,20:30,Hoffenheim,RB Leipzig,1
167,2024-04-27,15:30,RB Leipzig,Dortmund,0
168,2024-05-04,15:30,Augsburg,Dortmund,2


In [74]:
merged = df_reduziert.merge(df_reduziert, left_on= ["datum","verein"], right_on=["datum","gegner"])

In [75]:
merged

,datum,uhrzeit_x,gegner_x,verein_x,predictions_x,uhrzeit_y,gegner_y,verein_y,predictions_y
0,2024-04-27,18:30,Leverkusen,Stuttgart,1,18:30,Stuttgart,Bayer Leverkusen,2
1,2024-05-04,15:30,Bayern München,Stuttgart,2,15:30,Stuttgart,Bayern Munich,0
2,2024-04-27,15:30,Dortmund,RB Leipzig,2,15:30,RB Leipzig,Dortmund,0
3,2024-05-03,20:30,Hoffenheim,RB Leipzig,1,20:30,RB Leipzig,Hoffenheim,2
4,2024-04-27,15:30,RB Leipzig,Dortmund,0,15:30,Dortmund,RB Leipzig,2
5,2024-05-04,15:30,Augsburg,Dortmund,2,15:30,Dortmund,Augsburg,1
6,2024-04-27,15:30,Wolfsburg,Freiburg,2,15:30,Freiburg,Wolfsburg,1
7,2024-05-04,18:30,Köln,Freiburg,1,18:30,Freiburg,Koln,2
8,2024-04-27,15:30,Werder Bremen,Augsburg,2,15:30,Augsburg,Werder Bremen,1
9,2024-05-04,15:30,Dortmund,Augsburg,1,15:30,Augsburg,Dortmund,2


In [77]:
merged[(merged["predictions_x"] == 2) & (merged["predictions_y"] == 0)].value_counts()

datum       uhrzeit_x  gegner_x        verein_x       predictions_x  uhrzeit_y  gegner_y       verein_y         predictions_y
2024-04-27  15:30      Dortmund        RB Leipzig     2              15:30      RB Leipzig     Dortmund         0                1
2024-05-04  15:30      Bayern München  Stuttgart      2              15:30      Stuttgart      Bayern Munich    0                1
                       M'Gladbach      Werder Bremen  2              15:30      Werder Bremen  Monchengladbach  0                1
2024-05-05  15:30      Bochum          Union Berlin   2              15:30      Union Berlin   Bochum           0                1
dtype: int64

In [78]:
merged[(merged["predictions_x"] == 1) & (merged["predictions_y"] == 1)].value_counts()

Series([], dtype: int64)